In [6]:
import pandas as pd

df = pd.read_csv('../out/SDS00001.CSV', skiprows=10, usecols=[3,4], names=['time','val'])

print(df)





          time   val
0     -0.26528  2.54
1     -0.26524  2.56
2     -0.26520  2.54
3     -0.26516  2.56
4     -0.26512  2.56
...        ...   ...
20467  0.55340  2.54
20468  0.55344  2.54
20469  0.55348  2.56
20470  0.55352  2.54
20471  0.55356  2.54

[20472 rows x 2 columns]


In [7]:
import plotly.express as px

fig = px.line(df, x="time", y="val", title='reviever out')
fig.show()

In [8]:

start_time = .119
end_time = .271

df1 = df[(df['time'] > start_time) & (df['time'] < end_time)]

fig = px.line(df1, x="time", y="val", title='reviever out')
fig.show()




In [9]:
import numpy as np


threshold = 1.5

diffs = df1['val'].diff()

edges = df1.index[np.abs(diffs) > threshold]

print(edges[0])
print(df1['val'].loc[edges[0]])

print(edges[1])
print(df1['val'].loc[edges[1]])

# print(diffs)

# fig = px.line( y=diffs, title='reviever out')
# fig.show()




# for i in range(32):






9622
0.36
9629
2.96


In [11]:
import sys  
sys.path.insert(0, '/home/axlan/src/sainlogic-sdr/gr-sainlogic/python')

from sainlogic_parser import check_crc

fs = 25e3
threshold = 1.5

aprox_symbol_duration = 0.0004625
aprox_symbol_samples = int(aprox_symbol_duration * fs)

num_bits = 128

bits = [True]

max_len = int(aprox_symbol_samples * 1.5)

threshold_data = df1['val'].values > threshold

while len(bits) < num_bits:
    neg_index = np.argmax(threshold_data != bits[-1])
    assert(neg_index != 0)
    assert(neg_index < max_len)
    next_idx = neg_index + max_len
    if len(bits) == 9 and not threshold_data[next_idx]:
        bits = [True] + bits
    bits.append(threshold_data[next_idx])
    threshold_data = threshold_data[next_idx:]

bits = [ int(i) for i in bits ]


print(bits)

a = np.array(bits)
rs = a.reshape((-1, 8))
print(rs)

bytes = []
for val in rs:
    binval = ''.join([str(i) for i in val])
    bytes.append(int(binval,2))


print(bytes)
print(''.join([ f'{i:02x}' for i in bytes  ]))

print(check_crc(bytes))




[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1]
[[1 1 1 1 1 1 1 1]
 [1 1 0 1 0 1 0 0]
 [1 1 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [1 0 0 1 1 0 1 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [1 0 0 0 0 1 0 0]
 [1 0 0 0 0 1 1 0]
 [0 0 1 0 1 0 0 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 0 1 1]
 [1 1 1 1 1 0 1 1]
 [0 1 0 0 0 1 0 1]]
[255, 212, 196, 0, 0, 0, 154, 0, 0, 132, 134, 41, 255, 251, 251, 69]
ffd4c40000009a0000848629fffbfb45
True
